Datasets/DataFrames Spark: Calculate Chi Squared values

In [1]:
val reviewsDF = spark.read.options(Map("header"->"true")).format("json").load("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json").select("category","reviewText")
reviewsDF.printSchema()

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_0171
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_0171)
SparkSession available as 'spark'


root
 |-- category: string (nullable = true)
 |-- reviewText: string (nullable = true)



reviewsDF: org.apache.spark.sql.DataFrame = [category: string, reviewText: string]


In [40]:
// Tokenize and Case Folding
import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.ml.util.DefaultParamsWritable
//import org.apache.spark.sql.functions.udf

class CustomTokenizer extends Tokenizer with DefaultParamsWritable {

  // use splitting pattern from exercise 1
  override protected def createTransformFunc: String => Seq[String] = { input =>
    input.toLowerCase.split("[^a-zA-Z<>^|]+").toSeq
  }
}

val tokenizer = new CustomTokenizer()
.setInputCol("reviewText")
.setOutputCol("words")

val tokenized = tokenizer.transform(reviewsDF).select("category","words")

//tokenized.show()

import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.ml.util.DefaultParamsWritable
defined class CustomTokenizer
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_c15664cdb2b5
tokenized: org.apache.spark.sql.DataFrame = [category: string, words: array<string>]


In [41]:
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

class CustomStopWordsRemover(stopWordsFile: String) extends StopWordsRemover {
    // load and set custom stop words
    val customStopWords: Array[String] = scala.io.Source.fromFile(stopWordsFile).getLines.toArray
    setStopWords(customStopWords)    
}


val stopWordsFile = "../data/stopwords.txt"
val remover = new CustomStopWordsRemover(stopWordsFile)
  .setInputCol("words")
  .setOutputCol("filtered")

val filtered = remover.transform(tokenized).select("category", "filtered")
//filtered.show()


// val query3 = filtered.filter(array_contains($"filtered", "ever"))
// query3.show()

import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
defined class CustomStopWordsRemover
stopWordsFile: String = stopwords.txt
remover: CustomStopWordsRemover = StopWordsRemover: uid=stopWords_83208a65f616, numStopWords=596, locale=en_US, caseSensitive=false
filtered: org.apache.spark.sql.DataFrame = [category: string, filtered: array<string>]


In [42]:
val tokenFreqByCategory = filtered
.withColumn("token", explode(array_distinct($"filtered")))
.groupBy("category", "token")
.count()
.withColumnRenamed("count", "A").orderBy(desc("A"))

//tokenFreqByCategory.show()

tokenFreqByCategory: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, token: string ... 1 more field]


In [43]:
import org.apache.spark.ml.feature.ChiSqSelector
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.stat.ChiSquareTest
import org.apache.spark.sql.functions._

// Step 1: for token t number of occurrens within each category -> A
val tokenFreqByCategory = filtered
 .withColumn("token", explode(array_distinct($"filtered")))
.groupBy("category", "token")
.count()
.withColumnRenamed("count", "A").orderBy(desc("A"))

//tokenFreqByCategory.show()

// Step 2: for token t total number of occurrencs across all categories -> B = this - A
val t_total_number_of_occurrences = tokenFreqByCategory
  .groupBy("token")
  .agg(sum("A").alias("total_number_of_occurrences")).orderBy(desc("total_number_of_occurrences"))
//t_total_number_of_occurrences.show()

// Step 3: number of reviews by category
val n_docs_by_cat = filtered.groupBy("category").agg(count("*").as("n_docs_by_cat")) // C = this - A

// Step 4: total number of reviews
val n_of_docs = n_docs_by_cat.agg(sum("n_docs_by_cat").alias("N")) // N
// join both dataframes
val crossjoin_n_info = n_docs_by_cat.crossJoin(n_of_docs)

import org.apache.spark.ml.feature.ChiSqSelector
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.stat.ChiSquareTest
import org.apache.spark.sql.functions._
tokenFreqByCategory: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, token: string ... 1 more field]
t_total_number_of_occurrences: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [token: string, total_number_of_occurrences: bigint]
n_docs_by_cat: org.apache.spark.sql.DataFrame = [category: string, n_docs_by_cat: bigint]
n_of_docs: org.apache.spark.sql.DataFrame = [N: bigint]
crossjoin_n_info: org.apache.spark.sql.DataFrame = [category: string, n_docs_by_cat: bigint ... 1 more field]


In [44]:
val chiSquaredValues = tokenFreqByCategory
.join(t_total_number_of_occurrences, ("token"))
.join(crossjoin_n_info, ("category"))
.withColumn("B", $"total_number_of_occurrences" - $"A")
.withColumn("C", $"n_docs_by_cat" - $"A")
.withColumn("D", $"N" - $"A" - $"B" - $"C")
.withColumn("D", $"N" - $"A" - $"B" - $"C")
.withColumn("chisquared",
  ($"N" * pow($"A" * $"D" - $"B" * $"C", 2)) /
    (($"A" + $"B") * ($"A" + $"C") * ($"B" + $"D") * ($"C" + $"D"))
)
.select("category", "token","chisquared").orderBy(desc("chisquared"))

// chiSquaredValues.show()

chiSquaredValues: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, token: string ... 1 more field]


In [67]:
// Top 75
import org.apache.spark.sql.expressions.Window
val windowSpec = Window.partitionBy("category").orderBy(desc("chisquared"))

val top75ByCategory = chiSquaredValues
.withColumn("rank", row_number().over(windowSpec))
.filter(col("rank") <= 75)
//.withColumn("value_str", col("chisquared").cast("string"))


top75ByCategory.show()

+----------+----------+------------------+----+
|  category|     token|        chisquared|rank|
+----------+----------+------------------+----+
|Automotive|       oem|1068.8583585543724|   1|
|Automotive|     honda|1035.2233546903822|   2|
|Automotive|    engine| 763.2772560433446|   3|
|Automotive|   vehicle| 667.9866189155996|   4|
|Automotive|headlights| 661.4206245337347|   5|
|Automotive|   exhaust| 627.5001547880793|   6|
|Automotive|      jeep| 613.5399313513781|   7|
|Automotive| installed| 556.7738877280015|   8|
|Automotive|      tire| 547.3958796305628|   9|
|Automotive| headlight| 522.1912239048576|  10|
|Automotive|    toyota| 469.6296601687105|  11|
|Automotive|   factory|  415.447649158483|  12|
|Automotive|    dealer| 413.4005822828253|  13|
|Automotive|dealership| 397.0329781505319|  14|
|Automotive|   muffler| 394.6383917024621|  15|
|Automotive|      ford| 381.6024077213795|  16|
|Automotive|     wiper|   376.08092581619|  17|
|Automotive|    nissan| 370.521825556082

import org.apache.spark.sql.expressions.Window
windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@434e325d
top75ByCategory: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, token: string ... 2 more fields]


In [ ]:
// category, [{token, chi_squared}]

val output = top75ByCategory
.groupBy("category")
.agg(collect_list(concat_ws( ",", $"token", $"chisquared")) as "top_tokens")
.withColumn("output", concat_ws(":", $"category", $"top_tokens"))
.select("output")

// Show the result
output.show()

output
.select("output")
.write
.format("text") // Change format to "text" for TXT file
.option("header", "false") // If you want to include headers, set this to true
.mode("overwrite") // Overwrite the output if it already exists, or use "append" or "ignore"
.save("output.txt")

+--------------------+
|              output|
+--------------------+
|Automotive:oem,10...|
|Baby:diaper,2429....|
|Book:reading,6184...|
|Clothing_Shoes_an...|
|Health_and_Person...|
|Kindle_Store:auth...|
|Movies_and_TV:dvd...|
|Apps_for_Android:...|
|Beauty:lotion,231...|
|CDs_and_Vinyl:mus...|
|Cell_Phones_and_A...|
|Digital_Music:mus...|
|Electronic:cable,...|
|Grocery_and_Gourm...|
|Home_and_Kitche:v...|
|Musical_Instrumen...|
|Office_Product:ca...|
|Patio_Lawn_and_Ga...|
|Pet_Supplie:cat,4...|
|Sports_and_Outdoo...|
+--------------------+
only showing top 20 rows



In [97]:
output
.select("output")
.write
.format("text") // Change format to "text" for TXT file
.option("header", "false") // If you want to include headers, set this to true
.mode("overwrite") // Overwrite the output if it already exists, or use "append" or "ignore"
.save("file:///home/dic24/e12239877/Exercise_2/output.txt")
 //"file:///path/to/output.txt"

<console>: 68: error: org.apache.spark.sql.DataFrameWriter[org.apache.spark.sql.Row] does not take parameters

In [101]:
output.rdd.saveAsTextFile("file:///home/dic24/e12239877/Exercise_2/output.txt")

org.apache.spark.SparkException:  Job aborted.